# First Random Forest Baseline

Target variable: Is fire present --> Multiclass classifier\
Target variable: Fire class --> Multiclass classifier\
Target variable: Fire size --> Regressor

Multi-class Classification Options:

* Random forest
* Multiclass logistic regression
* SVM
* K-NN
* Gradient boosting


Regression Options:

* Random forecast regressor
* Multivariate regression
* Lasso regression
* Ridge regression

In [82]:
import pickle
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [46]:
from sklearn.ensemble import RandomForestClassifier 
# Parameters: (n_estimators=100)

from sklearn.linear_model import LogisticRegression 
# Parameters: (penalty={‘l1’, ‘l2’, ‘elasticnet’, ‘none’}, C=1.0)

from sklearn.svm import SVC 
# Parameters: (C=1.0, kernel={‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’})

from sklearn.neighbors import KNeighborsClassifier 
# Parameters: (n_neighbors=5, weights='uniform', leaf_size=30, p=2)

from sklearn.ensemble import GradientBoostingClassifier
# Parameters: (learning_rate=0.1, n_estimators=100

from sklearn.ensemble import RandomForestRegressor 
# Parameters: (n_estimators=100)

from sklearn.linear_model import LinearRegression
# No parameters

from sklearn.linear_model import Lasso
# Parameters: (alpha=1.0)

from sklearn.linear_model import Ridge
# Parameters: (alpha=1.0)

In [68]:
# Import main df
directory = 'C:/Users/jades/1001 Intro to Data Science Notebooks/Project/wildfires-1001/data/clean_data/'

df = pd.read_csv(directory + 'target_df_final_geo.csv', index_col=0)

# Import column renaming dictionary
dic_file_path = 'C:/Users/jades/1001 Intro to Data Science Notebooks/Project/wildfires-1001/data/clean_data/ERA_weather-data/ERA_rename_dictionary.pkl'

with open(dic_file_path, 'rb') as handle:
    dic = pickle.load(handle)

# Rename columns
df = df.rename(columns=dic)

In [72]:
# Create the dataframes for our features and for each of the target variables
# Drop the unneccessary columns 
non_mod_cols = ['GRID_ID',
 'month_id',
 'MONTH',
 'YEAR',
 'COUNTYFP',
 'NAME',
 'GRID_AREA',
 'COUNTY_ARE',
 'month_id_old_x',
 'month_id_old_y',
 'geometry',
 'Y_bin',
# 'Y_fire_count',
# 'Y_fire_area_prop',
 'Y_fire_class_size']

df_train = df[df['YEAR'] <= 2017]
df_test = df[df['YEAR'] > 2017]

train_X = df_train.drop(non_mod_cols, axis=1)

test_X = df_test.drop(non_mod_cols, axis=1)

train_Y_bin = df_train['Y_bin']
#train_Y_area = df_train['Y_fire_area_prop']
#train_Y_count = df_train['Y_fire_count']
train_Y_class = df_train['Y_fire_class_size']

test_Y_bin = df_test['Y_bin']
#test_Y_count = df_test['Y_fire_count']
#test_Y_area = df_test['Y_fire_area_prop']
test_Y_class = df_test['Y_fire_class_size']

In [74]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(train_X, train_Y_bin)

RandomForestClassifier()

In [83]:
train_X.dtypes

elev_mean                 float64
elev_std                  float64
elev_media                float64
elev_max                  float64
elev_min                  float64
elev_range                float64
pl_count                  float64
total_pl_length           float64
road_count                float64
total_road_length         float64
MEDIAN_AGE_TOT            float64
MEDIAN_AGE_MALE           float64
MEDIAN_AGE_FEM            float64
AGEUNDER13_TOT            float64
AGE1424_TOT               float64
AGE2544_TOT               float64
AGE4564_TOT               float64
AGE65PLUS_TOT             float64
COUNTY_AREA               float64
POPDENSITY                float64
POPDENSITY_MALE           float64
POPDENSITY_FEM            float64
Structure Arsons            int64
Mobile Arsons               int64
Other Arsons                int64
Total Arsons                int64
Total Arsons Cleared        int64
Unemployment              float64
medianHHI2018               int64
adj_fire_count

In [85]:
np.where(train_X.values >= np.finfo(np.float32).max)

(array([], dtype=int64), array([], dtype=int64))